In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from google.colab import drive
drive.mount('/content/drive')

!pip install scikit-learn==0.23.2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
product_des=pd.read_csv('/content/drive/MyDrive/Home_depot_relevance_search/product_descriptions.csv')
product_des.shape


product_des.isnull().sum()

product_uid            0
product_description    0
dtype: int64

In [4]:

product_des.shape
product_des.head()

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...
4,100005,Update your bathroom with the Delta Vero Singl...


In [5]:
product_des1=product_des.head(10000)
product_des1.shape
import sklearn
print('sklearn',sklearn.__version__)


sklearn 0.23.2


In [6]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import re
import string
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
stopwords=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def clean_text(txt):
  txt="".join([c for c in txt if c not in string.punctuation])
  tokens=re.split('\W+',txt)
  txt=[word for word in tokens if word not in stopwords]
  txt=[lemmatizer.lemmatize(word) for word in txt if word not in stopwords]
  return txt

In [8]:
product_des1['product_description_2']=product_des1['product_description'].apply(lambda x:clean_text(x))
product_des1.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,product_uid,product_description,product_description_2
0,100001,"Not only do angles make joints stronger, they ...","[Not, angle, make, joint, stronger, also, prov..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...,"[BEHR, Premium, Textured, DECKOVER, innovative..."
2,100003,Classic architecture meets contemporary design...,"[Classic, architecture, meet, contemporary, de..."
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...,"[The, Grape, Solar, 265Watt, Polycrystalline, ..."
4,100005,Update your bathroom with the Delta Vero Singl...,"[Update, bathroom, Delta, Vero, SingleHandle, ..."


In [9]:
def lemmatization(input_text):
  text=[lemmatizer.lemmatize(word)for word in input_text if word not in stopwords]
  return text

In [10]:
product_des1['product_description_2']=product_des1['product_description_2'].apply(lambda x:lemmatization(x))



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
data=product_des1['product_description_2']
corpus=[]
for line in data:
  description=' '.join(line)
  description=description.lower()
  corpus.append(description)



corpus

['not angle make joint stronger also provide consistent straight corner simpson strongtie offer wide variety angle various size thickness handle lightduty job project structural connection needed some bent skewed match project for outdoor project moisture present use zmax zinccoated connector provide extra resistance corrosion look z end model numberversatile connector various 90 connection home repair projectsstronger angled nailing screw fastening alonehelp ensure joint consistently straight strongdimensions 3 x 3 x 112 inmade 12gauge steelgalvanized extra corrosion resistanceinstall 10d common nail 9 x 112 strongdrive sd screw',
 'behr premium textured deckover innovative solid color coating it bring old weathered wood concrete back life the advanced 100 acrylic resin formula creates durable coating tired worn deck rejuvenating whole new look for best result sure properly prepare surface using applicable behr product displayed abovecalifornia resident seenbspproposition 65 informati

#Writing corpus.txt file

In [12]:
with open('corpus.txt', 'w') as filehandle:
    for listitem in corpus:
        filehandle.write('%s\n' % listitem)

#Reading corpus.txt file

In [13]:
corpus_data = []

# open file and read the content in a list
with open('corpus.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        currentPlace = line[:-1]

        # add item to the list
        corpus_data.append(line)
     

In [14]:
corpus_data


['not angle make joint stronger also provide consistent straight corner simpson strongtie offer wide variety angle various size thickness handle lightduty job project structural connection needed some bent skewed match project for outdoor project moisture present use zmax zinccoated connector provide extra resistance corrosion look z end model numberversatile connector various 90 connection home repair projectsstronger angled nailing screw fastening alonehelp ensure joint consistently straight strongdimensions 3 x 3 x 112 inmade 12gauge steelgalvanized extra corrosion resistanceinstall 10d common nail 9 x 112 strongdrive sd screw\n',
 'behr premium textured deckover innovative solid color coating it bring old weathered wood concrete back life the advanced 100 acrylic resin formula creates durable coating tired worn deck rejuvenating whole new look for best result sure properly prepare surface using applicable behr product displayed abovecalifornia resident seenbspproposition 65 informa

#Feature extraction from product Description

In [15]:
vectorizer=TfidfVectorizer(stop_words='english',analyzer='word',max_features=500)


#X=vectorizer.fit_transform(converted_list)
X=vectorizer.fit_transform(corpus_data)
X

<10000x500 sparse matrix of type '<class 'numpy.float64'>'
	with 287011 stored elements in Compressed Sparse Row format>

#Visualizing Clusters in subset of Data

In [16]:
#from sklearn.cluster import KMeans
#within_cluster_sum_of_sqr=[]

#for i in range(1,11):
  #km=KMeans(n_clusters=i,
#            init='k-means++',
#            max_iter=300,
#            n_init=10,
#            random_state=0,
#            algorithm='full',
#            tol=0.001)
#  km.fit(X)
#  labels=km.labels_
#  within_cluster_sum_of_sqr.append(km.inertia_)

In [17]:
#plt.rcParams['figure.figsize']=(13,6)
#plt.plot(range(1,11),within_cluster_sum_of_sqr)
#plt.grid()
#plt.tight_layout()
#plt.title('The Elbow Method')

In [18]:
#kmeans=KMeans(n_clusters=10,init='k-means++')
#y_kmeans=kmeans.fit_predict(X)
#plt.plot(y_kmeans,".")
#plt.show()

In [19]:
def print_cluster(i):
  cluster_list=[]
  cluster_list.clear()
  print("Cluster %d:" % i)
  for ind in ordered_centroids[i, :10]:
    print(' %s' % terms[ind])
    cluster_list.append(terms[ind])

  return cluster_list
          

In [20]:
k_value=10

model=KMeans(n_clusters=k_value ,
             init='k-means++',
             max_iter=300,
             n_init=10,
             random_state=0,
             algorithm='full',
            tol=0.001
             )
             
model.fit(X)

  

KMeans(algorithm='full', n_clusters=10, random_state=0, tol=0.001)

In [21]:
print("Top search per clusters :")
ordered_centroids=model.cluster_centers_.argsort()[:,::-1]
terms=vectorizer.get_feature_names()
for i in range(k_value):
  print_cluster(i)

Top search per clusters :
Cluster 0:
 use
 ft
 easy
 steel
 used
 design
 feature
 designed
 product
 outdoor
Cluster 1:
 fan
 air
 control
 cooking
 filter
 heat
 oven
 help
 ft
 room
Cluster 2:
 paint
 wood
 color
 rug
 vary
 surface
 65
 seenbspproposition
 resident
 stain
Cluster 3:
 battery
 power
 lithiumion
 charger
 tool
 compact
 protection
 plan
 impact
 depot
Cluster 4:
 tile
 piece
 floor
 wall
 flooring
 indoor
 commercial
 installation
 residential
 case
Cluster 5:
 door
 glass
 hinge
 security
 steel
 opening
 wood
 panel
 easy
 lock
Cluster 6:
 light
 bulb
 led
 lighting
 fixture
 energy
 incandescent
 lamp
 glass
 shade
Cluster 7:
 blade
 cutting
 saw
 cut
 steel
 handle
 tool
 speed
 motor
 feature
Cluster 8:
 water
 valve
 pipe
 heater
 faucet
 toilet
 tank
 hot
 pressure
 fitting
Cluster 9:
 shelf
 cabinet
 storage
 drawer
 vanity
 finish
 sink
 hardware
 design
 faucet


In [22]:
ordered_centroids.shape

(10, 500)

In [23]:
import pickle

pickle.dump(model,open('Kmeans_cluster.pkl','wb'))
pickle.load(open('Kmeans_cluster.pkl','rb'))

KMeans(algorithm='full', n_clusters=10, random_state=0, tol=0.001)

In [24]:
model1=pickle.load(open('Kmeans_cluster.pkl','rb'))

In [25]:
def selected_cluster(i):
  cluster=[]
  cluster.clear()
  print("Cluster %d:" % i)
  for ind in ordered_centroids[i, :10]:
    print(' %s' % terms[ind])
    cluster.append(terms[ind])
  return cluster


In [26]:
def show_recommendations(product):
  Y=vectorizer.transform([product])
  prediction=model1.predict(Y)
  cluster=selected_cluster(prediction[0])
  return cluster

In [27]:
domain=show_recommendations("cutting tool")

Cluster 7:
 blade
 cutting
 saw
 cut
 steel
 handle
 tool
 speed
 motor
 feature


In [28]:
domain

['blade',
 'cutting',
 'saw',
 'cut',
 'steel',
 'handle',
 'tool',
 'speed',
 'motor',
 'feature']

In [29]:
#joined_df= pd.read_csv('/content/drive/MyDrive/Home_depot_relevance_search/joined_prod_title_and_Prod_des.csv',
                       #encoding= 'unicode_escape')

#joined_df= pd.read_csv('https://drive.google.com/file/d/1ayy2Qr-DzdHZrmKKm4PC-iQ0CcmLaQ54/view?usp=sharing',
                       #encoding= 'unicode_escape')

import pandas as pd

url = 'https://drive.google.com/file/d/1ayy2Qr-DzdHZrmKKm4PC-iQ0CcmLaQ54/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
joined_df = pd.read_csv(path,encoding= 'unicode_escape')

In [30]:
joined_df=joined_df.dropna()
joined_df

,product_uid,product_description,product_title
0,100001,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle
1,100001,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie 12-Gauge Angle
2,100002,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...
3,100005,Update your bathroom with the Delta Vero Singl...,Delta Vero 1-Handle Shower Only Faucet Trim Ki...
4,100005,Update your bathroom with the Delta Vero Singl...,Delta Vero 1-Handle Shower Only Faucet Trim Ki...
...,...,...,...
74062,206638,"Atlantic, Inc. 94835722 Uniquely designed for ...",Atlantic Windowpane 576 CD or 192 DVD Blu-Ray ...
74063,206639,Philips Energy Advantage lamps use less energy...,Philips 40-Watt Halogen R20 Flood Light Bulb (...
74064,206641,The Schlage Camelot In-Active Aged Bronze Hand...,Schlage Camelot In-Active Aged Bronze Handlese...
74065,206648,The Rose Garden is inspired by the popular ear...,Plastec 11 in. x 24 in. Rose Garden Wall Decor...


In [31]:
new_df=joined_df[joined_df['product_title'].str.contains('lithium-ion',regex=False,case=False,na=False)]

In [32]:
products=new_df['product_title'].unique()
print(products[:10])
products=pd.DataFrame(products)
products.head(10)



['Milwaukee M18 FUEL 18-Volt Brushless Lithium-Ion Hole Hawg 1/2 in. Right Angle Drill (Bare Tool)'
 'RIDGID X4 18-Volt Lithium-Ion Cordless Drill and Impact Driver Combo Kit (2-Tool)'
 'DEWALT 20-Volt Max Lithium-Ion Cordless Combo Kit (9-Tool)'
 'DEWALT 20-Volt 3Ah Max Lithium-Ion Battery (2-Pack)'
 'Milwaukee M12 12-Volt Lithium-Ion Compact Battery (2-Pack)'
 'Milwaukee M18 Fuel 18-Volt Lithium-Ion Brushless Cordless 1/2 in. High Torque Impact Wrench with Friction Ring Kit'
 'Milwaukee M12 FUEL 12-Volt Lithium-Ion 5-3/8 in. Cordless Circular Saw'
 'Ryobi QuickTurn 4-Volt Lithium-Ion 1/4 in. Cordless Screwdriver'
 'DEWALT 20-Volt Max Lithium-Ion 1/2 in. High Torque Impact Wrench with Detent Pin (Tool-Only)'
 'DEWALT 20-Volt MAX Lithium-Ion Cordless Combo Kit with Tough Case (4-Tool)']


,0
0,Milwaukee M18 FUEL 18-Volt Brushless Lithium-I...
1,RIDGID X4 18-Volt Lithium-Ion Cordless Drill a...
2,DEWALT 20-Volt Max Lithium-Ion Cordless Combo ...
3,DEWALT 20-Volt 3Ah Max Lithium-Ion Battery (2-...
4,Milwaukee M12 12-Volt Lithium-Ion Compact Batt...
5,Milwaukee M18 Fuel 18-Volt Lithium-Ion Brushle...
6,Milwaukee M12 FUEL 12-Volt Lithium-Ion 5-3/8 i...
7,Ryobi QuickTurn 4-Volt Lithium-Ion 1/4 in. Cor...
8,DEWALT 20-Volt Max Lithium-Ion 1/2 in. High To...
9,DEWALT 20-Volt MAX Lithium-Ion Cordless Combo ...


In [33]:
df=pd.DataFrame(products[:10])
df.rename({0:"Featuring top 10 Products"},axis=1,inplace=True)
df

,Featuring top 10 Products
0,Milwaukee M18 FUEL 18-Volt Brushless Lithium-I...
1,RIDGID X4 18-Volt Lithium-Ion Cordless Drill a...
2,DEWALT 20-Volt Max Lithium-Ion Cordless Combo ...
3,DEWALT 20-Volt 3Ah Max Lithium-Ion Battery (2-...
4,Milwaukee M12 12-Volt Lithium-Ion Compact Batt...
5,Milwaukee M18 Fuel 18-Volt Lithium-Ion Brushle...
6,Milwaukee M12 FUEL 12-Volt Lithium-Ion 5-3/8 i...
7,Ryobi QuickTurn 4-Volt Lithium-Ion 1/4 in. Cor...
8,DEWALT 20-Volt Max Lithium-Ion 1/2 in. High To...
9,DEWALT 20-Volt MAX Lithium-Ion Cordless Combo ...


In [34]:
#from sklearn.metrics import silhouette_score
#for n_clusters in range(2,11):
    #clusterer = KMeans(n_clusters=n_clusters)
    #preds = clusterer.fit_predict(X)
    #centers = clusterer.cluster_centers_

    #score = silhouette_score(X, preds)
    #print("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))